## This notebook contains pieces of code which is used to generate figures.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LogNorm, TwoSlopeNorm
import os

import astropy.io.fits as fits
import hcipy

from pastis.simulators.scda_telescopes import HexRingAPLC
import pastis.util as util
from pastis.config import CONFIG_PASTIS

os.chdir('../ULTRA')
from config import CONFIG_ULTRA
from ultra.util import sort_1d_mus_per_actuator, sort_1d_mus_per_seg

#### Create data directory

In [ ]:
data_dir = CONFIG_PASTIS.get('local', 'local_data_path')
resDir = util.create_data_path(data_dir, 'figures')
os.makedirs(resDir, exist_ok=True)


## Generate optical schematics for a telescope simulator

#### For SCDA Designs

In [ ]:
optics_dir = os.path.join(util.find_repo_location(), 'data', 'SCDA')
NUM_RINGS = 5
sampling = 4

tel = HexRingAPLC(optics_dir, NUM_RINGS, sampling)
psf, intermediates = tel.calc_psf(display_intermediate=False, 
                                  return_intermediate='intensity',
                                  norm_one_photon=True)

### Plot intensities are various intermediate planes

In [ ]:
fpm_mask = np.zeros(len(intermediates['after_fpm']))

for i in range(0, len(intermediates['after_fpm'])):
    if intermediates['after_fpm'][i] == 0.0:
        fpm_mask[i] = 0
    else:
        fpm_mask[i] = 1

In [ ]:
plt.figure(figsize = (10, 7))

# Entrance Pupil
plt.subplot(2, 3, 1)
hcipy.imshow_field(tel.apodizer, mask = tel.aperture, cmap='Greys_r')
plt.tick_params(bottom=False, left=False, labelleft=False, labelbottom=False)
#plt.colorbar()


# before FPM
plt.subplot(2, 3, 2)
hcipy.imshow_field(intermediates['before_fpm'], 
                   norm=LogNorm(vmin=1e-8, vmax=1e-1), cmap='inferno')
plt.tick_params(bottom=False, left=False, labelleft=False, labelbottom=False)
#plt.colorbar()

# after FPM, note: calc_psf returns int_after_fpm in log scale.
plt.subplot(2, 3, 3)
hcipy.imshow_field(10**(intermediates['after_fpm']), 
                   norm=LogNorm(vmin=1e-8, vmax=1e-1),mask = fpm_mask,cmap= 'inferno')
plt.tick_params(bottom=False, left=False, labelleft=False, labelbottom=False)
#plt.colorbar()


# before Lyot Stop
plt.subplot(2, 3, 4)
hcipy.imshow_field(intermediates['before_lyot'], norm=LogNorm(vmin=1e-3, vmax=1), 
                   cmap='inferno')
plt.tick_params(bottom=False, left=False, labelleft=False, labelbottom=False)
#plt.colorbar()

# after Lyot Stop
plt.subplot(2, 3, 5)
hcipy.imshow_field(intermediates['after_lyot'], mask=tel.lyotstop, norm=LogNorm(vmin=1e-3, vmax=1),
                   cmap='inferno')
plt.tick_params(bottom=False, left=False, labelleft=False, labelbottom=False)
#plt.colorbar()

# final PSF
plt.subplot(2, 3, 6)
hcipy.imshow_field(psf, norm=LogNorm(vmin=1e-14, vmax=1e-3), cmap='inferno')
plt.tick_params(bottom=False, left=False, labelleft=False, labelbottom=False)
#plt.colorbar()

plt.tight_layout()
plt.savefig(os.path.join(resDir, f'optical_train_{NUM_RINGS}_rings.png'))






# Generate pm/s surface maps 


### Get necessary data files / initialize variables.

In [ ]:
#Check the NUM_RINGS for the following data path.
NUM_RINGS = 5

local_data_path = CONFIG_ULTRA.get('local_path', 'local_data_path')
mus_csv_data_path = os.path.join(local_data_path, '2023-10-03T16-48-52_hexringtelescope/mus_Hex_5_1e-11.csv') 

optimal_wavescale = 10     # get this value from tolerance_table2.txt in mus_csv_data_path folder
fractional_scale = 0.0001  # get this value from tolerance_table2.txt from same folder

mus = np.genfromtxt(mus_csv_data_path, delimiter=',')

In [ ]:
optics_dir = os.path.join(util.find_repo_location(), 'data', 'SCDA')
sampling = 4
tel = HexRingAPLC(optics_dir, NUM_RINGS, sampling)

WHICH_DM = 'harris_seg_mirror'

if WHICH_DM == 'harris_seg_mirror':
    fpath = CONFIG_PASTIS.get('LUVOIR', 'harris_data_path')  # path to Harris spreadsheet
    pad_orientations = np.pi / 2 * np.ones(CONFIG_PASTIS.getint('LUVOIR', 'nb_subapertures'))
    DM_SPEC = (fpath, pad_orientations, True, False, False)
    NUM_MODES = 5
    
tel.create_segmented_harris_mirror(fpath, pad_orientations, thermal=True, mechanical=False, other=False)

mus_per_actuator = sort_1d_mus_per_actuator(mus, NUM_MODES, tel.nseg)  # in nm

mu_maps = []

for mode in range(NUM_MODES):
    coeffs = mus_per_actuator[mode]
    if WHICH_DM == 'harris_seg_mirror':
        tel.harris_sm.actuators = coeffs * 1e-9 / 2  # in meters of surface
        mu_maps.append(tel.harris_sm.surface)  # in m
    if WHICH_DM == 'seg_mirror':
        tel.sm.actuators = coeffs * 1e-9 / 2  # in meters of surface
        mu_maps.append(tel.sm.surface)  # in m

In [ ]:
plt.figure(figsize=(32, 5))
plt.subplot(1, 5, 1)
#plot_norm = TwoSlopeNorm(vcenter=0, vmin=-5, vmax=5)
plot_norm = TwoSlopeNorm(vcenter=0)
hcipy.imshow_field((mu_maps[0]) * 1e12 * np.sqrt(fractional_scale * optimal_wavescale ** 2), norm=plot_norm, cmap='RdBu')
plt.tick_params(top=False, bottom=True, left=True, right=False, labelleft=True, labelbottom=True, labelsize=15)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=15)
cbar.set_label("Surface (pm/s)", fontsize=15)
plt.tight_layout()

plt.subplot(1, 5, 2)
#plot_norm = TwoSlopeNorm(vcenter=0, vmin=-10, vmax=10)
plot_norm = TwoSlopeNorm(vcenter=0, vmin=-0.75, vmax=0.75)
hcipy.imshow_field((mu_maps[1]) * 1e12 * np.sqrt(fractional_scale * optimal_wavescale ** 2), norm=plot_norm, cmap='RdBu')
plt.tick_params(top=False, bottom=True, left=True, right=False, labelleft=True, labelbottom=True, labelsize=15)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=15)
cbar.set_label("Surface (pm/s)", fontsize=15)
plt.tight_layout()

plt.subplot(1, 5, 3)
#plot_norm = TwoSlopeNorm(vcenter=0, vmin=-8, vmax=8)
plot_norm = TwoSlopeNorm(vcenter=0)
hcipy.imshow_field((mu_maps[2]) * 1e12 * np.sqrt(fractional_scale * optimal_wavescale ** 2), norm=plot_norm, cmap='RdBu')
plt.tick_params(top=False, bottom=True, left=True, right=False, labelleft=True, labelbottom=True, labelsize=15)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=15)
cbar.set_label("Surface (pm/s)", fontsize=15)
plt.tight_layout()

plt.subplot(1, 5, 4)
#plot_norm = TwoSlopeNorm(vcenter=0, vmin=-7, vmax=7)
plot_norm = TwoSlopeNorm(vcenter=0)
hcipy.imshow_field((mu_maps[3]) * 1e12 * np.sqrt(fractional_scale * optimal_wavescale ** 2), norm=plot_norm, cmap='RdBu')
plt.tick_params(top=False, bottom=True, left=True, right=False, labelleft=True, labelbottom=True, labelsize=15)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=15)
cbar.set_label("Surface (pm/s)", fontsize=15)
plt.tight_layout()

plt.subplot(1, 5, 5)
#plot_norm = TwoSlopeNorm(vcenter=0, vmin=-12, vmax=12)
plot_norm = TwoSlopeNorm(vcenter=0)
hcipy.imshow_field((mu_maps[4]) * 1e12 * np.sqrt(fractional_scale * optimal_wavescale ** 2), norm=plot_norm, cmap='RdBu')
plt.tick_params(top=False, bottom=True, left=True, right=False, labelleft=True, labelbottom=True, labelsize=15)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=15)
cbar.set_label("Surface (pm/s)", fontsize=15)
plt.tight_layout()
plt.savefig(os.path.join(resDir, f'surface_pm_s_{NUM_RINGS}.png'))

# Plot tolerance maps in units of mK

#### Note: why we multiplying 1000 to luvoir.sm.surface; recall the harris modal basis are essentially surface deformation formed when there is  a 1mk temperature change across different directions. 
Let $\mathcal{f(x,y)}$ be the harris modal basis function.
At any point $\mathcal(x,y)$ on the segment, $\mathcal{f(x,y)}$ denotes the surface deformation in pm. 
<br> Each segment in the figure named 'stat_mu_map_..' represents $\mu_{kk}* \mathcal{f(x,y)}$. 
where $\mu_{kk}$ is an element from $\textit{mu_map_harris}$ and it's unit is in nm. 
<br> Therefore, $10^3*\mu_{kk}*f(x , y)$ is in pm scale.
<br> $f(x,y)$ corresponds to 1 mk temperature change across the second.
<br> $10^3*\mu_{kk}*f(x , y)$ will correspond to $10^3*\mu_{kk}$ mK temperature change.
<br> Hence temperature-tolerances per mode per segment is $10^3*\mu_{kk}$ mK.
<br> In the just above codeblock, we are feeding each actuators of tel.sm.actuators with $\mu_{kk}$. 
<br> Therefore, we multiply by a factor of 1000 i.e, to make it $10^3*\mu_{kk}$ and see the plots in mK scale

In [ ]:
mus_per_segment = sort_1d_mus_per_seg(mus, NUM_MODES, tel.nseg)

tel.create_segmented_mirror(1)  # setting each segment to deform only in piston 

plt.figure(figsize=(32, 5))

#for Harris modes use the following titles
plot_titles = ['Faceplate Silvered', 'Bulk', 'Gradient Radial', 'Gradient X Lateral', 'Gradient Z axial']

v_maxs = [1, 3, 6, 2, 2]  # use this for 1_hex
# v_maxs = [1, 5, 8, 3, 3]  # for 2_hex
# v_maxs = [0.5, 7, 6, 2, 2]  # for 3_hex
# v_maxs = [0.5, 7, 6, 2, 2]  # for 4_hex
# v_maxs = [0.5, 7, 6, 2, 2]  # for 5_hex

for mode in range(NUM_MODES):
    plt.subplot(1, NUM_MODES, mode+1)
    plt.title(plot_titles[mode], fontsize=15)
    tel.sm.flatten()
    tel.sm.actuators = mus_per_segment[mode]
    hcipy.imshow_field((tel.sm.surface)*1000, cmap = 'YlOrRd', vmin=0, vmax = v_maxs[mode])
    plt.tick_params(top =False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
    cbar=plt.colorbar()
    cbar.ax.tick_params(labelsize = 15)
    cbar.set_label("mK",fontsize = 15)
    plt.tight_layout()
    plt.savefig(os.path.join(resDir, f'surface_mK_{NUM_RINGS}.png'))

# Plot tolerance maps in units of mK/s 

In [ ]:
plt.figure(figsize=(32, 5))

#for Harris modes use the following titles
plot_titles = ['Faceplate Silvered', 'Bulk', 'Gradient Radial', 'Gradient X Lateral', 'Gradient Z axial']

# v_maxs = [0.5, 1.5, 2, 1, 1]  # use this for 1_hex
# v_maxs = [0.3, 1.5, 2, 0.8, 0.8]  # for 2_hex
# v_maxs = [0.06, 0.5, 0.5, 0.2, 0.2]  # for 3_hex
# v_maxs = [0.06, 0.8, 0.5, 0.3, 0.2]  # for 4_hex
v_maxs = [0.08, 1.0, 0.7, 0.3, 0.2]  # for 5_hex

for mode in range(NUM_MODES):
    plt.subplot(1, NUM_MODES, mode+1)
    #plt.title(plot_titles[mode], fontsize=15)
    tel.sm.flatten()
    tel.sm.actuators = mus_per_segment[mode]
    hcipy.imshow_field((tel.sm.surface) * 1000 * np.sqrt(fractional_scale * optimal_wavescale ** 2), 
                       cmap = 'YlOrRd', vmin=0, vmax=v_maxs[mode])
    plt.tick_params(top =False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
    cbar=plt.colorbar()
    cbar.ax.tick_params(labelsize = 15)
    cbar.set_label("mK/s",fontsize = 15)
    plt.tight_layout()
    plt.savefig(os.path.join(resDir, f'surface_mK_s_{NUM_RINGS}.png'))

# Plot contrast vs t_exp, for different scda designs.  

In [ ]:
# Bring all the analysis run for all scda designs to a common folder, give the path to 'data_path'

data_path = ''
hex1_data = '2023-10-02T17-18-48_hexringtelescope'
hex2_data = '2023-10-03T14-03-30_hexringtelescope'
hex3_data = '2023-10-03T15-43-44_hexringtelescope'
hex4_data = '2023-10-03T16-24-08_hexringtelescope'
hex5_data = '2023-10-03T16-48-52_hexringtelescope'

TimeMinus = CONFIG_ULTRA.getfloat('close_loop', 'TimeMinus')
TimePlus = CONFIG_ULTRA.getfloat('close_loop', 'TimePlus')
Ntimes = CONFIG_ULTRA.getint('close_loop', 'Ntimes')

texp = np.logspace(TimeMinus, TimePlus, Ntimes)

C_TARGET = CONFIG_ULTRA.getfloat('target', 'contrast')
Vmag = CONFIG_ULTRA.getfloat('target', 'Vmag')
    
hex1 = np.genfromtxt(os.path.join(data_path, hex1_data,'contrast_wf.csv'), delimiter=',')
hex2 = np.genfromtxt(os.path.join(data_path, hex2_data,'contrast_wf.csv'), delimiter=',')
hex3 = np.genfromtxt(os.path.join(data_path, hex3_data,'contrast_wf.csv'), delimiter=',')
hex4 = np.genfromtxt(os.path.join(data_path, hex4_data,'contrast_wf.csv'), delimiter=',')
hex5 = np.genfromtxt(os.path.join(data_path, hex5_data,'contrast_wf.csv'), delimiter=',')

contrast_floor1 = 5.40e-11
contrast_floor2 = 4.17e-11
contrast_floor3 = 6.26e-11
contrast_floor4 = 8.90e-11
contrast_floor5 = 3.98e-11

plt.figure(figsize=(15, 10))

font = {'family': 'serif', 'color': 'black', 'weight': 'normal', 'size': 20}
plt.title('Target contrast = %s, Vmag= %s' % (C_TARGET, Vmag), fontdict=font)

#see config and tolerance table to know the optimal wavescale and accrodingly chose part of the hex data array.  
plt.plot(texp, hex1[5 * Ntimes:(5 + 1) * Ntimes] - contrast_floor1, 
         label=r'1Hex, $ Q = % .2f\ pm/s$' % (0.7841)) 
plt.plot(texp, hex2[0 * Ntimes:(0 + 1) * Ntimes]  - contrast_floor2, 
         label=r'2Hex, $ Q = % .2f\ pm/s$' % (0.5241))
plt.plot(texp, hex3[0 * Ntimes:(0 + 1) * Ntimes] - contrast_floor3, 
         label=r'3Hex, $ Q = % .2f\ pm/s$' % (0.5556))
plt.plot(texp, hex4[0 * Ntimes:(0 + 1) * Ntimes] - contrast_floor4, 
         label=r'4Hex, $ Q = % .2f\ pm/s$' % (0.6457))
plt.plot(texp, hex5[0 * Ntimes:(0 + 1) * Ntimes] - contrast_floor5, 
         label=r'5Hex, $ Q= % .2f\ pm/s$' % (0.9226))


plt.xlabel("$t_{WFS}$ in secs", fontsize=20)
plt.ylabel(r"$ \Delta $ contrast", fontsize=20)
plt.yscale('log')
plt.xscale('log')
plt.legend(loc='upper center', fontsize=20)
plt.tick_params(top=True, bottom=True, left=True, right=True, labelleft=True, labelbottom=True, labelsize=20)
plt.tick_params(axis='both', which='major', length=10, width=2)
plt.tick_params(axis='both', which='minor', length=6, width=2)
plt.grid()
plt.tight_layout()
plt.savefig(os.path.join(resDir, f'temporal_designs.png'))